# 概率语法的模糊测试

来源：[Probabilistic Grammar Fuzzing](https://www.fuzzingbook.org/html/ProbabilisticGrammarFuzzer.html)

我们可以赋予文法更大的能力。比如，我们给每个表达式添加概率。这允许我们控制每个表达式的扩展概率，从而可以导向的测试特定的功能。我们也可以从用例输入中，提取这样的概率，并专门针对这些样本中不常见的输入特征进行测试。

有趣的东西：[本福特定律](https://zh.wikipedia.org/wiki/%E6%9C%AC%E7%A6%8F%E7%89%B9%E5%AE%9A%E5%BE%8B)
> 本福特定律说明在b进位制中，以数n起头的数出现的概率


In [ ]:
from fuzzingbook.fuzzingbook_utils.GrammarFuzzer import GrammarFuzzer, all_terminals, display_tree
from fuzzingbook.fuzzingbook_utils.Grammars import is_valid_grammar, EXPR_GRAMMAR, START_SYMBOL, crange, is_nonterminal,extend_grammar
from fuzzingbook.fuzzingbook_utils.Grammars import opts, exp_string, exp_opt, set_opts
from fuzzingbook.fuzzingbook_utils.Parser import Parser, EarleyParser, PEGParser
import random

## 给语法指定概率

给文法的表达式中添加概率：有两种情况：概率已经指定；概率没有指定，则为（100%-已经消耗的概率）/没有指定概率的表达式个数

In [ ]:
PROBABILISTIC_EXPR_GRAMMAR = {
    "<start>":
        ["<expr>"],

    "<expr>":
        [("<term> + <expr>", opts(prob=0.1)),
         ("<term> - <expr>", opts(prob=0.2)),
         "<term>"],

    "<term>":
        [("<factor> * <term>", opts(prob=0.1)),
         ("<factor> / <term>", opts(prob=0.1)),
         "<factor>"
         ],

    "<factor>":
        ["+<factor>", "-<factor>", "(<expr>)",
            "<leadinteger>", "<leadinteger>.<integer>"],

    "<leadinteger>":
        ["<leaddigit><integer>", "<leaddigit>"],

    # Benford's law: frequency distribution of leading digits
    "<leaddigit>":
        [("1", opts(prob=0.301)),
         ("2", opts(prob=0.176)),
         ("3", opts(prob=0.125)),
         ("4", opts(prob=0.097)),
         ("5", opts(prob=0.079)),
         ("6", opts(prob=0.067)),
         ("7", opts(prob=0.058)),
         ("8", opts(prob=0.051)),
         ("9", opts(prob=0.046)),
         ],

    # Remaining digits are equally distributed
    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
}

In [ ]:
assert is_valid_grammar(PROBABILISTIC_EXPR_GRAMMAR, supported_opts={'prob'})

In [ ]:
def exp_prob(expansion):
    # 返回给定表达式的概率
    return exp_opt(expansion, attribute='prob')

In [ ]:
def set_prob(grammar, symbol, expansion, prob):
    # 给指定的expansion指定概率
    set_opts(grammar, symbol, expansion, opts(prob=prob))

In [ ]:
exp_prob(PROBABILISTIC_EXPR_GRAMMAR["<leaddigit>"][0])

## 计算概率
获取给定文法规则的概率。检查一致性。

In [ ]:
def exp_probabilities(expansions, nonterminal="<symbol>"):
    # 检查概率和是否为1；
    # 将expansion和概率，建立映射。
    probabilities = [exp_prob(expansion) for expansion in expansions]
    prob_dist = prob_distribution(probabilities, nonterminal)

    prob_mapping = {}
    for i in range(len(expansions)):
        expansion = exp_string(expansions[i])
        prob_mapping[expansion] = prob_dist[i]

    return prob_mapping


def prob_distribution(probabilities, nonterminal="<symbol>"):
    # 检查概率和是否为1。对于缺省的概率，计算出来。
    # 返回expansions的概率。
    epsilon = 0.00001

    number_of_unspecified_probabilities = probabilities.count(None)
    if number_of_unspecified_probabilities == 0:
        assert abs(sum(probabilities) - 1.0) < epsilon, \
            nonterminal + ": sum of probabilities must be 1.0"
        return probabilities

    sum_of_specified_probabilities = 0.0
    for p in probabilities:
        if p is not None:
            sum_of_specified_probabilities += p
    assert 0 <= sum_of_specified_probabilities <= 1.0, \
        nonterminal + ": sum of specified probabilities must be between 0.0 and 1.0"

    default_probability = ((1.0 - sum_of_specified_probabilities)
                           / number_of_unspecified_probabilities)
    all_probabilities = []
    for p in probabilities:
        if p is None:
            p = default_probability
        all_probabilities.append(p)

    assert abs(sum(all_probabilities) - 1.0) < epsilon
    return all_probabilities


def is_valid_probabilistic_grammar(grammar, start_symbol=START_SYMBOL):
    # 检查文法的合法性：文法的合法性+概率的一致性
    if not is_valid_grammar(grammar, start_symbol):
        return False

    for nonterminal in grammar:
        expansions = grammar[nonterminal]
        _ = exp_probabilities(expansions, nonterminal)

    return True

## 在节点扩展选择中考虑概率因素

GrammarFuzzer中，在树结构中，随机选择一个叶子节点进行扩展。

扩展初期，使用最大代价的expansion进行扩展；我们从最大代价相同的expansion的list中，使用概率作为随机选择的权重，进行随机选择。

扩展中期，使用随机的expansion进行扩展；我们从中，选择使用概率作为随机选择的权重，进行随机选择。

扩展后期，使用最小代价的expansion进行扩展；我们从最小代价相同的expansion的list中，使用概率作为随机选择的权重，进行随机选择。

In [ ]:
class ProbabilisticGrammarFuzzer(GrammarFuzzer):
    def check_grammar(self):
        super().check_grammar()
        assert is_valid_probabilistic_grammar(self.grammar)

    def supported_opts(self):
        return super().supported_opts() | {'prob'}
    
    def choose_node_expansion(self, node, possible_children):
        (symbol, tree) = node
        expansions = self.grammar[symbol]
        probabilities = exp_probabilities(expansions)

        weights = []
        for child in possible_children:
            expansion = all_terminals((node, child)) # 这里挺好。将树结构存储的孩子节点，变回文法中的字符串结构。
            child_weight = probabilities[expansion]
            if self.log:
                print(repr(expansion), "p =", child_weight)
            weights.append(child_weight)

        if sum(weights) == 0:
            # No alternative (probably expanding at minimum cost)
            weights = None

        return random.choices(
            range(len(possible_children)), weights=weights)[0]

In [ ]:
leaddigit_fuzzer = ProbabilisticGrammarFuzzer(
    PROBABILISTIC_EXPR_GRAMMAR, start_symbol="<leaddigit>")
trials = 10000

count = {}
for c in crange('0', '9'):
    count[c] = 0

for i in range(trials):
    count[leaddigit_fuzzer.fuzz()] += 1

print([(digit, count[digit] / trials) for digit in count])

## 从用例中学习概率

从输入用例中，计算每个expansion出现的概率。（这里的expansion包含两种：语法中直接出现的expansion；经过推到生长的分析树的叶子节点串成的字符串）

In [ ]:
def expansion_key(symbol, expansion):
    """Convert (symbol, expansion) into a key.  `expansion` can be an expansion string or a derivation tree."""
    if isinstance(expansion, tuple):
        expansion = expansion[0]
    if not isinstance(expansion, str):
        children = expansion
        expansion = all_terminals((symbol, children))
    return symbol + " -> " + expansion

In [ ]:
class ExpansionCountMiner(object):
    def __init__(self, parser, log=False):
        assert isinstance(parser, Parser)
        self.grammar = extend_grammar(parser.grammar())
        self.parser = parser
        self.log = log
        self.reset()

    def reset(self):
        self.expansion_counts = {}

    def add_coverage(self, symbol, children):
        key = expansion_key(symbol, children)

        if self.log:
            print("Found", key)

        if key not in self.expansion_counts:
            self.expansion_counts[key] = 0
        self.expansion_counts[key] += 1


    def add_tree(self, tree):
        (symbol, children) = tree
        if not is_nonterminal(symbol):
            return

        direct_children = [
            (symbol, None) if is_nonterminal(symbol) else (
                symbol, []) for symbol, c in children]
        self.add_coverage(symbol, direct_children)

        for c in children:
            self.add_tree(c)

    def count_expansions(self, inputs):
        for inp in inputs:
            tree, *_ = self.parser.parse(inp)
            self.add_tree(tree)

    def counts(self):
        return self.expansion_counts

In [ ]:
def decrange(start, end):
    """Return a list with string representations of numbers in the range [start, end)"""
    return [repr(n) for n in range(start, end)]

IP_ADDRESS_GRAMMAR = {
    "<start>": ["<address>"],
    "<address>": ["<octet>.<octet>.<octet>.<octet>"],
    # ["0", "1", "2", ..., "255"]
    "<octet>": decrange(0, 256)
}

expansion_count_miner = ExpansionCountMiner(EarleyParser(IP_ADDRESS_GRAMMAR))

In [ ]:
inputs = ["127.0.0.1", "1.2.3.4"]
expansion_count_miner.count_expansions(inputs)
expansion_count_miner.counts()

## 将学习出来的概率赋值到文法中

In [ ]:
class ProbabilisticGrammarMiner(ExpansionCountMiner):
    def set_probabilities(self, counts):
        for symbol in self.grammar:
            self.set_expansion_probabilities(symbol, counts)

    def set_expansion_probabilities(self, symbol, counts):
        expansions = self.grammar[symbol]
        if len(expansions) == 1:
            set_prob(self.grammar, symbol, expansions[0], None)
            return

        expansion_counts = [
            counts.get(
                expansion_key(
                    symbol,
                    expansion),
                0) for expansion in expansions]
        total = sum(expansion_counts)
        for i, expansion in enumerate(expansions):
            p = expansion_counts[i] / total if total > 0 else None
            # if self.log:
            #     print("Setting", expansion_key(symbol, expansion), p)
            set_prob(self.grammar, symbol, expansion, p)

    def mine_probabilistic_grammar(self, inputs):
        self.count_expansions(inputs)
        self.set_probabilities(self.counts())
        return self.grammar

In [ ]:
probabilistic_grammar_miner = ProbabilisticGrammarMiner(
    EarleyParser(IP_ADDRESS_GRAMMAR))
inputs=["127.0.0.1", "1.2.3.4"]
probabilistic_ip_address_grammar = probabilistic_grammar_miner.mine_probabilistic_grammar(inputs)
assert is_valid_probabilistic_grammar(probabilistic_ip_address_grammar)

# 通过学习一个样本，我们可以根据这个样本的(语法)属性调整模糊。
probabilistic_ip_fuzzer = ProbabilisticGrammarFuzzer(
    probabilistic_ip_address_grammar)
[probabilistic_ip_fuzzer.fuzz() for i in range(10)]

现在让我们来看看我们的三个使用场景。

第一个场景是直接从样本中创建概率分布，并在测试生成过程中使用这些分布。这有助于将测试生成集中在那些最常用的特性上，从而将客户遇到失败的风险降到最低。

第二个场景，我们也可以测试不常见的特性。也就是说，在我们的使用示例中很少出现的特性。这是安全测试中常见的场景，其中关注的是不常见的(可能是不太知名的)特性，因为更少的用户意味着报告的bug更少，因此有更多的bug有待发现和利用。

第三个场景，我们也可以从输入的子集中学习集中于该子集中出现的特性(或者相反地，避免其特性)。例如，如果我们知道，有一些包含感兴趣的功能的输入子集(例如，因为它特别重要，或者因为它最近被更改了)。我们可以从这个子集中学习，并将测试生成集中在它的特性上。